<a href="https://colab.research.google.com/github/merttezcan/fakenews-twitter/blob/main/fakenews_twitter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install dgl -f https://data.dgl.ai/wheels/repo.html

Looking in links: https://data.dgl.ai/wheels/repo.html
     |████████████████████████████████| 5.5 MB 2.6 MB/s 


In [4]:
import pandas as pd
import numpy as np
import dgl
import torch as th

In [5]:
#loading the nodes and features, replacing ids with new consecutive ids starting from zero.
#also doing some preprocessing

nodes_data = pd.read_csv('gilani-2017-features.csv')
nodes_data.rename(columns={'id': 'tw_id'}, inplace=True)
nodes_data.insert(0, 'new_id', range(0, 0 + len(nodes_data)))
nodes_data.drop(['name','screen_name','url','lang','time_zone','location','default_profile',
                           'default_profile_image','geo_enabled','profile_image_url','profile_use_background_image',
                           'profile_background_image_url_https','profile_text_color','profile_image_url_https',
                           'profile_sidebar_border_color','profile_background_tile','profile_sidebar_fill_color',
                           'profile_background_image_url','profile_background_color','profile_link_color','utc_offset',
                           'description','created_at'], axis=1, inplace=True)

nodes_data.replace({'class': {'human': True, 'bot': False}}, inplace=True)

display(nodes_data)

,new_id,class,tw_id,statuses_count,followers_count,friends_count,favourites_count,listed_count,is_translator,follow_request_sent,protected,verified,notifications,contributors_enabled,following
0,0,False,461277906,10728,379139,2949,1,3406,False,False,False,True,False,False,False
1,1,False,808280162,141143,26048,337,26670,32,False,False,False,False,False,False,False
2,2,False,1575510320,180426,120558,108607,454,79,False,False,False,False,False,False,False
3,3,False,2449235599,13046,26258,18,3889,1,False,False,True,False,False,False,False
4,4,False,1113769100,72434,75208,78570,13374,35,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2413,2413,True,68765193,8730,941859,858,25,525,False,False,False,True,False,False,False
2414,2414,True,782496626451546112,1,4,0,0,0,False,False,False,False,False,False,False
2415,2415,True,134164413,56525,923657,51269,261,2667,False,False,False,False,False,False,False
2416,2416,True,72378123,29992,852471,115447,4875,979,False,False,False,True,False,False,False


In [6]:
#creating a small df to have tw_id and new_id, then converting it to a dictionary

df= nodes_data[['tw_id', 'new_id']]
df = df.astype('int64')
id_dict = dict(df.values)

In [103]:
#checking our dict

#print(id_dict)

In [8]:
#loading the friends file that contains user id and friend id

edges_data = pd.read_csv("friends.csv",names=['user_id', 'friend_id'])
edges_data.dropna(inplace=True)

edges_data = edges_data.astype('int64')

print(edges_data)

           user_id            friend_id
0        461277906  1093701661292355584
1        461277906           2250033132
2        461277906            124955314
3        461277906   931101388716650496
4        461277906           1113629671
...            ...                  ...
1483155  150711216            120977182
1483156  150711216             44652080
1483157  150711216             92344997
1483158  150711216             66930860
1483159  150711216            109771764

[1482748 rows x 2 columns]


In [9]:
#using the dict that we created before, matching the ids with the new ones

edges_data['fr_source'] = edges_data['user_id'].map(id_dict)
edges_data['fr_target'] = edges_data['friend_id'].map(id_dict)


display(edges_data)

,user_id,friend_id,fr_source,fr_target
0,461277906,1093701661292355584,0.0,NaN
1,461277906,2250033132,0.0,NaN
2,461277906,124955314,0.0,NaN
3,461277906,931101388716650496,0.0,NaN
4,461277906,1113629671,0.0,NaN
...,...,...,...,...
1483155,150711216,120977182,2417.0,NaN
1483156,150711216,44652080,2417.0,NaN
1483157,150711216,92344997,2417.0,NaN
1483158,150711216,66930860,2417.0,NaN


In [10]:
#dropping the null values, that means we are removing the users that are not connected to each other

fr_edges_df = edges_data.dropna()
fr_edges_df = fr_edges_df.astype('int64')

In [11]:
display(fr_edges_df)

,user_id,friend_id,fr_source,fr_target
3726,973224960,777983107,5,2256
3936,973224960,111809956,5,394
4075,973224960,355784391,5,818
4102,973224960,19478383,5,2088
4371,194908659,50393960,6,1542
...,...,...,...,...
1480977,134164413,171337448,2415,2209
1482832,150711216,116611790,2417,2218
1482945,150711216,135236273,2417,2219
1483092,150711216,78039816,2417,861


In [12]:
#doing same process for the followers file

edges_data2 = pd.read_csv("followers.csv",names=['user_id', 'follower_id'])
edges_data2.dropna(inplace=True)

edges_data2 = edges_data2.astype('int64')

print(edges_data2)

           user_id          follower_id
0        461277906  1363871922677706752
1        461277906  1359030742030848000
2        461277906           1116957792
3        461277906  1363863589677801472
4        461277906           1492422193
...            ...                  ...
2147336  150711216           2351018816
2147337  150711216  1166382282019618816
2147338  150711216  1213583487162208256
2147339  150711216  1163515847358451712
2147340  150711216            594134386

[2146803 rows x 2 columns]


In [13]:
edges_data2['fw_source'] = edges_data2['user_id'].map(id_dict)
edges_data2['fw_target'] = edges_data2['follower_id'].map(id_dict)


display(edges_data2)

,user_id,follower_id,fw_source,fw_target
0,461277906,1363871922677706752,0.0,NaN
1,461277906,1359030742030848000,0.0,NaN
2,461277906,1116957792,0.0,NaN
3,461277906,1363863589677801472,0.0,NaN
4,461277906,1492422193,0.0,NaN
...,...,...,...,...
2147336,150711216,2351018816,2417.0,NaN
2147337,150711216,1166382282019618816,2417.0,NaN
2147338,150711216,1213583487162208256,2417.0,NaN
2147339,150711216,1163515847358451712,2417.0,NaN


In [14]:
fw_edges_df = edges_data2.dropna()
fw_edges_df = fw_edges_df.astype('int64')

In [15]:
display(fw_edges_df)

,user_id,follower_id,fw_source,fw_target
3011,1113769100,2363987964,4,645
18255,218697713,15040988,19,976
20434,73725383,547977614,21,20
30592,1570434660,506437283,33,1877
30838,1570434660,759460784,33,2386
...,...,...,...,...
1992115,777983107,491090439,2256,1004
2027143,107930490,15485441,2292,1554
2040987,162173169,148753334,2306,1103
2091236,262749118,22411342,2360,920


In [101]:
# Create an empty list
fr_list =[]

# Iterate over each row
for rows in fr_edges_df.itertuples():
	# Create list for the current row
	my_list =[rows.fr_source, rows.fr_target]
	
	# append the list to the final list
	fr_list.append(my_list)

# Print the list
#print(fr_list)

In [102]:
# Create an empty list
fw_list =[]

# Iterate over each row
for rows in fw_edges_df.itertuples():
	# Create list for the current row
	my_list =[rows.fw_source, rows.fw_target]
	
	# append the list to the final list
	fw_list.append(my_list)

# Print the list
#print(fw_list)

In [18]:
#creating an heterograph with using our edge lists

g = dgl.heterograph(
    {('user', 'friend', 'user'): fr_list,
     ('user', 'follow', 'user'): fw_list
     })
g

Graph(num_nodes={'user': 2418},
      num_edges={('user', 'follow', 'user'): 533, ('user', 'friend', 'user'): 7809},
      metagraph=[('user', 'user', 'follow'), ('user', 'user', 'friend')])

In [19]:
g['friend'].edges()

(tensor([   5,    5,    5,  ..., 2417, 2417, 2417]),
 tensor([2256,  394,  818,  ..., 2219,  861,  443]))

In [20]:
#a function for creating tensors for features easily

def tensorCreator(df):

  i = 1

  for column in df.columns[1:]:
    globals()["t"+ str(i)] = th.tensor(df[column].values)
    i += 1

tensorCreator(nodes_data)   

In [21]:
#adding the tensors to the graph

g.ndata['class'] = t1
g.ndata['tw_id'] = t2
g.ndata['statuses_count'] = t3
g.ndata['followers_count'] = t4
g.ndata['friends_count'] = t5
g.ndata['favourites_count'] = t6
g.ndata['listed_count'] = t7
g.ndata['is_translator'] = t8
g.ndata['follow_request_sent'] = t9
g.ndata['protected'] = t10
g.ndata['verified'] = t11
g.ndata['notifications'] = t12
g.ndata['contributors_enabled'] = t13
g.ndata['following'] = t14

In [22]:
#done! we can check all of the nodes' features. for example:
g.nodes[2400]

NodeSpace(data={'class': tensor([True]), 'tw_id': tensor([282006208]), 'statuses_count': tensor([25336]), 'followers_count': tensor([3663561]), 'friends_count': tensor([638]), 'favourites_count': tensor([111]), 'listed_count': tensor([1185]), 'is_translator': tensor([False]), 'follow_request_sent': tensor([False]), 'protected': tensor([False]), 'verified': tensor([True]), 'notifications': tensor([False]), 'contributors_enabled': tensor([False]), 'following': tensor([False])})

In [23]:
!pip install node2vec

  Created wheel for node2vec: filename=node2vec-0.4.3-py3-none-any.whl size=5978 sha256=df63ef20285dc6faf7a87907a49151b4e02f8dfd235622cbb3549e16e651aace
  Stored in directory: /root/.cache/pip/wheels/07/62/78/5202cb8c03cbf1593b48a8a442fca8ceec2a8c80e22318bae9
Successfully built node2vec


In [24]:
import networkx as nx
from node2vec import Node2Vec

In [25]:
KGF = nx.Graph(fr_list)

In [26]:
KGF.number_of_nodes(), KGF.number_of_edges()

(1764, 7314)

In [27]:
KGFol = nx.Graph(fw_list)

In [28]:
KGFol.number_of_nodes(), KGFol.number_of_edges()

(532, 522)

In [29]:
node2vec = Node2Vec(KGF, dimensions=32, walk_length=30, num_walks=200, workers=4)

Computing transition probabilities:   0%|          | 0/1764 [00:00<?, ?it/s]

In [30]:
model = node2vec.fit(window=10, min_count=1, batch_words=4)

In [31]:
node2vec_fw = Node2Vec(KGFol, dimensions=32, walk_length=30, num_walks=200, workers=4)

Computing transition probabilities:   0%|          | 0/532 [00:00<?, ?it/s]

In [32]:
model_fw = node2vec_fw.fit(window=10, min_count=1, batch_words=4)

In [33]:
arr = []
arr2 = []
for i in range(0, KGF.number_of_nodes()):
  arr.append(model.wv.vectors[i])
  arr2.append(i)

In [34]:
n_dict = dict(zip(arr2, arr))

In [35]:
print(len(n_dict))

1764


In [36]:
#check
print(model.wv.vectors[7])
print(n_dict[7])

[ 0.07906826 -0.16399223  0.06040652 -0.09636401  0.14311644 -0.4989346
  0.46026725  0.4372431  -0.02379883 -0.16923895 -0.00941227 -0.31730112
 -0.04579119 -0.01536021  0.30664122 -0.23522836 -0.08835968  0.14635283
 -0.6472149  -0.39647296  0.06061589 -0.20517573 -0.1996076  -0.07191047
 -0.4191811   0.13527091  0.08147043 -0.15008108 -0.4261992  -0.46359593
 -0.159444   -0.2019969 ]
[ 0.07906826 -0.16399223  0.06040652 -0.09636401  0.14311644 -0.4989346
  0.46026725  0.4372431  -0.02379883 -0.16923895 -0.00941227 -0.31730112
 -0.04579119 -0.01536021  0.30664122 -0.23522836 -0.08835968  0.14635283
 -0.6472149  -0.39647296  0.06061589 -0.20517573 -0.1996076  -0.07191047
 -0.4191811   0.13527091  0.08147043 -0.15008108 -0.4261992  -0.46359593
 -0.159444   -0.2019969 ]


In [37]:
fw_arr = []
fw_arr2 = []
for i in range(0, KGFol.number_of_nodes()):
    fw_arr.append(model_fw.wv.vectors[i])
    fw_arr2.append(i)

In [38]:
n_dict_fw = dict(zip(fw_arr2, fw_arr))

In [39]:
#check
print(model_fw.wv.vectors[500])
print(n_dict_fw[500])

[ 0.45380005 -1.0902637  -0.29382357  0.66595083 -0.034418   -0.10894652
 -0.6283923  -0.47171608  1.1709592   0.62895566  0.933724   -0.651086
 -0.64669394  0.03058423 -0.6966853   0.17684942  0.9196023  -0.07306619
 -0.47049367 -0.06013809  0.30282485 -0.4528522   0.16792718  0.2860608
  0.6807774  -0.2482059  -0.23634876 -0.03607553 -0.16557486  0.73983693
 -0.09461167 -0.08649144]
[ 0.45380005 -1.0902637  -0.29382357  0.66595083 -0.034418   -0.10894652
 -0.6283923  -0.47171608  1.1709592   0.62895566  0.933724   -0.651086
 -0.64669394  0.03058423 -0.6966853   0.17684942  0.9196023  -0.07306619
 -0.47049367 -0.06013809  0.30282485 -0.4528522   0.16792718  0.2860608
  0.6807774  -0.2482059  -0.23634876 -0.03607553 -0.16557486  0.73983693
 -0.09461167 -0.08649144]


In [40]:
embedding_df_fr = pd.DataFrame.from_dict(n_dict, orient='index').reset_index()

In [41]:
col_list = []
for i in range (0,32):
    col_list.append("f" + str(i))
    
col_list.insert(0, "id")

embedding_df_fr.columns = col_list

In [42]:
list2 = col_list.copy()

In [43]:
list2.remove('id')

In [44]:
display(embedding_df_fr)

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0,-0.117201,0.409337,0.426590,0.228631,0.274507,-0.229006,0.124316,0.049839,-0.094654,-0.014116,-0.154381,-0.166307,-0.348701,-0.326775,0.077049,-0.379721,-0.439038,0.453245,-0.041057,-0.507203,-0.084443,0.203825,-0.246984,0.056289,0.053008,0.086311,0.002480,-0.236196,0.036224,-0.116992,-0.340265,-0.234395
1,1,0.385979,0.193495,0.126181,-0.002087,0.024265,-0.597926,-0.070327,0.198867,0.159160,-0.349507,-0.250878,-0.098472,-0.217339,0.196565,0.420611,-0.076537,-0.223275,0.046849,-0.027016,-0.438679,0.171973,-0.260298,-0.271796,-0.059343,0.350097,0.211243,-0.354090,-0.490547,-0.113013,-0.203750,-0.082388,-0.418740
2,2,-0.064782,0.108189,0.127211,-0.250686,0.042157,-0.471815,-0.080546,0.506772,-0.450210,0.294858,-0.529802,-0.337239,0.035277,-0.038158,0.140384,-0.605187,-0.066688,0.054905,-0.242787,-0.169881,0.266806,-0.129055,-0.291298,0.127108,-0.342228,-0.001519,0.094846,-0.141385,-0.016041,-0.131881,-0.021937,-0.411880
3,3,-0.076317,0.080320,0.151600,-0.345150,-0.291232,-0.511972,0.075673,0.224371,0.057458,0.103038,0.005813,-0.591917,-0.118435,-0.345642,0.380310,-0.321145,-0.185318,0.666653,-0.281543,0.109790,0.160803,0.135809,-0.443835,0.125477,-0.351253,0.345872,-0.130948,0.437441,0.039912,0.070868,-0.208381,-0.217727
4,4,-0.157010,-0.067280,-0.073825,-0.362780,0.316503,-0.570813,-0.309009,0.321580,-0.187550,-0.079376,-0.267249,0.070483,-0.541142,-0.419533,0.488073,-0.477435,0.201581,0.198375,0.108074,0.294739,0.433275,0.012902,-0.314358,-0.227786,-0.136518,0.149573,0.375444,0.126721,0.408518,-0.436911,-0.464190,0.119823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,1759,-0.465017,0.038992,-0.442403,-0.220012,0.392921,0.046240,-0.064529,0.910962,0.220765,0.645999,0.157324,-0.679245,-0.126590,-0.597919,1.230929,-0.498219,0.838593,-0.440109,0.100059,-0.659324,-0.243971,0.068949,-0.340660,-0.250752,-0.105393,0.889527,-0.161363,-0.503576,-0.285870,-0.452497,0.020703,0.267032
1760,1760,-0.548569,-0.663030,0.168672,-0.641771,0.007498,-0.427982,0.378094,-0.061272,-0.427714,0.461209,0.707848,-0.271527,-0.249382,-0.598250,0.266436,-0.582289,0.005334,0.768157,-0.534579,-0.163336,-0.270858,0.117708,-0.957364,0.464780,0.120735,0.173431,0.449871,0.132667,0.075446,-0.061919,-0.928581,-0.535764
1761,1761,-0.906810,0.033241,0.099167,-0.534468,-0.551261,-0.903424,-0.007098,0.303231,-0.251554,0.577695,0.149808,-0.825780,-0.332948,0.081355,0.120388,-0.389623,0.262438,0.472770,0.232752,-0.013581,0.100448,0.190852,-0.097274,-0.235218,0.083243,0.479284,0.463104,0.405101,-0.541159,-0.928349,-0.491973,-0.507083
1762,1762,0.015028,-0.035452,0.167666,-0.131281,-0.372677,-0.074276,-0.036944,0.210421,-0.630576,0.022820,-0.138178,-0.516788,0.050625,-0.006030,0.660804,-0.731573,0.023167,0.576400,0.128687,-0.046356,0.023050,0.394018,-0.318206,0.694861,0.357953,0.677647,-0.371419,0.338643,-0.216629,-0.223026,-0.738672,-0.188209


In [45]:
embedding_df_fw = pd.DataFrame.from_dict(n_dict_fw, orient='index').reset_index()

In [46]:
embedding_df_fw.columns = col_list

In [47]:
display(embedding_df_fw)

,id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0,0.067297,-0.854068,0.022617,0.375430,0.251917,-0.582875,-0.620217,-0.069266,0.888907,0.460836,0.504129,0.406231,-0.392317,0.146413,-0.583869,-0.329916,-0.168720,-0.334211,-0.364756,-0.590663,-0.125161,0.268982,0.103953,0.504504,-0.005671,-0.035253,-0.377056,-0.163230,-0.371823,0.188064,-0.124672,-0.241982
1,1,-0.295130,0.114292,0.501191,-0.037883,-0.417940,-0.396932,-0.420840,-0.459975,-0.073341,0.536578,0.630999,-0.480174,0.670393,-0.967260,-0.138775,0.233076,0.794044,0.343860,0.055928,-0.991456,-0.899046,0.332245,-0.889232,0.508095,-0.037579,0.088525,0.883061,-0.461665,0.371669,0.384552,-0.238962,-0.196044
2,2,-0.358434,-0.205629,0.339129,0.059243,-0.004142,-0.875642,-0.292206,-0.289198,-0.141128,0.186835,0.278765,-0.234924,-0.326722,-0.810146,-0.143150,-0.591756,0.860537,-0.150910,-0.588586,-0.571522,-0.027770,-0.179085,-0.020719,-0.055240,-0.840412,-0.294705,0.057517,-0.811228,0.136582,-0.334811,-0.423009,0.106553
3,3,-0.451933,-0.177029,0.291225,0.845742,0.392633,-0.879141,-0.503915,-0.777574,0.580584,0.075442,0.071220,0.245301,0.178889,-0.201981,-0.435350,0.152941,0.668506,0.381373,0.065307,-0.285107,-0.169761,0.212478,-0.453010,0.101276,0.123662,0.088305,-0.156269,0.094405,0.112211,0.705911,-0.422329,0.002886
4,4,0.437141,-0.972768,-0.245991,0.736577,-0.077884,-0.314063,-0.791396,-0.433821,1.381226,0.541327,1.114046,-0.822030,-0.623057,0.172623,-0.540395,0.205880,0.696476,0.019181,-0.293866,-0.066941,0.179169,-0.250014,-0.085208,-0.076986,0.461671,-0.292519,-0.092738,0.058113,0.029575,0.479984,0.134014,-0.039524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,527,-0.392587,-0.238111,0.183387,0.025544,0.244128,-1.019061,-0.304388,-0.201951,-0.257089,0.254881,0.339217,-0.186791,-0.256550,-0.547393,0.328473,-0.665266,0.986272,-0.133061,-0.563681,-0.501052,-0.133284,-0.164493,-0.250689,0.065903,-0.561510,-0.236128,-0.006899,-0.886779,0.173936,-0.415270,-0.611317,0.145590
528,528,-0.024774,-1.357596,0.678190,-0.158537,0.254983,-1.106894,-0.300961,-0.059885,-0.655043,0.240861,-0.128521,-0.259077,-0.596088,-0.664933,0.254482,0.516710,1.376992,-0.617254,0.399254,-0.483847,-0.191858,0.605685,-0.006023,0.563271,-0.782622,-0.393992,0.647694,-0.645549,0.019915,-0.845144,-0.936064,-0.240237
529,529,-0.011030,-0.714760,0.465864,-0.566421,0.334325,-0.665347,0.138886,0.098305,0.336219,0.610640,-0.373839,-0.813104,-0.353800,-0.594263,-0.344744,-0.584396,1.146376,-0.672182,0.277512,-0.843296,0.093535,0.042552,-0.248951,0.241723,-0.871139,-0.073964,-0.140152,-1.298483,-0.019027,-0.197488,-0.648804,-0.020970
530,530,0.057146,-0.850629,0.856476,-0.070725,-0.359525,-0.828572,-0.455994,-0.510129,-0.365962,0.194220,0.040094,-0.269874,-0.578357,-0.205123,0.379241,0.096772,1.145650,-0.250697,0.102737,-0.507992,0.309758,-0.141527,0.196611,0.565100,-0.871661,0.145145,0.513386,-1.219248,-0.177590,-0.192106,-0.974852,0.262908


In [48]:
embedding_df_fr.rename(columns={'id': 'new_id'}, inplace=True)

In [49]:
display(embedding_df_fr)

,new_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0,-0.117201,0.409337,0.426590,0.228631,0.274507,-0.229006,0.124316,0.049839,-0.094654,-0.014116,-0.154381,-0.166307,-0.348701,-0.326775,0.077049,-0.379721,-0.439038,0.453245,-0.041057,-0.507203,-0.084443,0.203825,-0.246984,0.056289,0.053008,0.086311,0.002480,-0.236196,0.036224,-0.116992,-0.340265,-0.234395
1,1,0.385979,0.193495,0.126181,-0.002087,0.024265,-0.597926,-0.070327,0.198867,0.159160,-0.349507,-0.250878,-0.098472,-0.217339,0.196565,0.420611,-0.076537,-0.223275,0.046849,-0.027016,-0.438679,0.171973,-0.260298,-0.271796,-0.059343,0.350097,0.211243,-0.354090,-0.490547,-0.113013,-0.203750,-0.082388,-0.418740
2,2,-0.064782,0.108189,0.127211,-0.250686,0.042157,-0.471815,-0.080546,0.506772,-0.450210,0.294858,-0.529802,-0.337239,0.035277,-0.038158,0.140384,-0.605187,-0.066688,0.054905,-0.242787,-0.169881,0.266806,-0.129055,-0.291298,0.127108,-0.342228,-0.001519,0.094846,-0.141385,-0.016041,-0.131881,-0.021937,-0.411880
3,3,-0.076317,0.080320,0.151600,-0.345150,-0.291232,-0.511972,0.075673,0.224371,0.057458,0.103038,0.005813,-0.591917,-0.118435,-0.345642,0.380310,-0.321145,-0.185318,0.666653,-0.281543,0.109790,0.160803,0.135809,-0.443835,0.125477,-0.351253,0.345872,-0.130948,0.437441,0.039912,0.070868,-0.208381,-0.217727
4,4,-0.157010,-0.067280,-0.073825,-0.362780,0.316503,-0.570813,-0.309009,0.321580,-0.187550,-0.079376,-0.267249,0.070483,-0.541142,-0.419533,0.488073,-0.477435,0.201581,0.198375,0.108074,0.294739,0.433275,0.012902,-0.314358,-0.227786,-0.136518,0.149573,0.375444,0.126721,0.408518,-0.436911,-0.464190,0.119823
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,1759,-0.465017,0.038992,-0.442403,-0.220012,0.392921,0.046240,-0.064529,0.910962,0.220765,0.645999,0.157324,-0.679245,-0.126590,-0.597919,1.230929,-0.498219,0.838593,-0.440109,0.100059,-0.659324,-0.243971,0.068949,-0.340660,-0.250752,-0.105393,0.889527,-0.161363,-0.503576,-0.285870,-0.452497,0.020703,0.267032
1760,1760,-0.548569,-0.663030,0.168672,-0.641771,0.007498,-0.427982,0.378094,-0.061272,-0.427714,0.461209,0.707848,-0.271527,-0.249382,-0.598250,0.266436,-0.582289,0.005334,0.768157,-0.534579,-0.163336,-0.270858,0.117708,-0.957364,0.464780,0.120735,0.173431,0.449871,0.132667,0.075446,-0.061919,-0.928581,-0.535764
1761,1761,-0.906810,0.033241,0.099167,-0.534468,-0.551261,-0.903424,-0.007098,0.303231,-0.251554,0.577695,0.149808,-0.825780,-0.332948,0.081355,0.120388,-0.389623,0.262438,0.472770,0.232752,-0.013581,0.100448,0.190852,-0.097274,-0.235218,0.083243,0.479284,0.463104,0.405101,-0.541159,-0.928349,-0.491973,-0.507083
1762,1762,0.015028,-0.035452,0.167666,-0.131281,-0.372677,-0.074276,-0.036944,0.210421,-0.630576,0.022820,-0.138178,-0.516788,0.050625,-0.006030,0.660804,-0.731573,0.023167,0.576400,0.128687,-0.046356,0.023050,0.394018,-0.318206,0.694861,0.357953,0.677647,-0.371419,0.338643,-0.216629,-0.223026,-0.738672,-0.188209


In [50]:
embedding_df_fw.rename(columns={'id': 'new_id'}, inplace=True)

In [51]:
display(embedding_df_fw)

,new_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0,0.067297,-0.854068,0.022617,0.375430,0.251917,-0.582875,-0.620217,-0.069266,0.888907,0.460836,0.504129,0.406231,-0.392317,0.146413,-0.583869,-0.329916,-0.168720,-0.334211,-0.364756,-0.590663,-0.125161,0.268982,0.103953,0.504504,-0.005671,-0.035253,-0.377056,-0.163230,-0.371823,0.188064,-0.124672,-0.241982
1,1,-0.295130,0.114292,0.501191,-0.037883,-0.417940,-0.396932,-0.420840,-0.459975,-0.073341,0.536578,0.630999,-0.480174,0.670393,-0.967260,-0.138775,0.233076,0.794044,0.343860,0.055928,-0.991456,-0.899046,0.332245,-0.889232,0.508095,-0.037579,0.088525,0.883061,-0.461665,0.371669,0.384552,-0.238962,-0.196044
2,2,-0.358434,-0.205629,0.339129,0.059243,-0.004142,-0.875642,-0.292206,-0.289198,-0.141128,0.186835,0.278765,-0.234924,-0.326722,-0.810146,-0.143150,-0.591756,0.860537,-0.150910,-0.588586,-0.571522,-0.027770,-0.179085,-0.020719,-0.055240,-0.840412,-0.294705,0.057517,-0.811228,0.136582,-0.334811,-0.423009,0.106553
3,3,-0.451933,-0.177029,0.291225,0.845742,0.392633,-0.879141,-0.503915,-0.777574,0.580584,0.075442,0.071220,0.245301,0.178889,-0.201981,-0.435350,0.152941,0.668506,0.381373,0.065307,-0.285107,-0.169761,0.212478,-0.453010,0.101276,0.123662,0.088305,-0.156269,0.094405,0.112211,0.705911,-0.422329,0.002886
4,4,0.437141,-0.972768,-0.245991,0.736577,-0.077884,-0.314063,-0.791396,-0.433821,1.381226,0.541327,1.114046,-0.822030,-0.623057,0.172623,-0.540395,0.205880,0.696476,0.019181,-0.293866,-0.066941,0.179169,-0.250014,-0.085208,-0.076986,0.461671,-0.292519,-0.092738,0.058113,0.029575,0.479984,0.134014,-0.039524
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
527,527,-0.392587,-0.238111,0.183387,0.025544,0.244128,-1.019061,-0.304388,-0.201951,-0.257089,0.254881,0.339217,-0.186791,-0.256550,-0.547393,0.328473,-0.665266,0.986272,-0.133061,-0.563681,-0.501052,-0.133284,-0.164493,-0.250689,0.065903,-0.561510,-0.236128,-0.006899,-0.886779,0.173936,-0.415270,-0.611317,0.145590
528,528,-0.024774,-1.357596,0.678190,-0.158537,0.254983,-1.106894,-0.300961,-0.059885,-0.655043,0.240861,-0.128521,-0.259077,-0.596088,-0.664933,0.254482,0.516710,1.376992,-0.617254,0.399254,-0.483847,-0.191858,0.605685,-0.006023,0.563271,-0.782622,-0.393992,0.647694,-0.645549,0.019915,-0.845144,-0.936064,-0.240237
529,529,-0.011030,-0.714760,0.465864,-0.566421,0.334325,-0.665347,0.138886,0.098305,0.336219,0.610640,-0.373839,-0.813104,-0.353800,-0.594263,-0.344744,-0.584396,1.146376,-0.672182,0.277512,-0.843296,0.093535,0.042552,-0.248951,0.241723,-0.871139,-0.073964,-0.140152,-1.298483,-0.019027,-0.197488,-0.648804,-0.020970
530,530,0.057146,-0.850629,0.856476,-0.070725,-0.359525,-0.828572,-0.455994,-0.510129,-0.365962,0.194220,0.040094,-0.269874,-0.578357,-0.205123,0.379241,0.096772,1.145650,-0.250697,0.102737,-0.507992,0.309758,-0.141527,0.196611,0.565100,-0.871661,0.145145,0.513386,-1.219248,-0.177590,-0.192106,-0.974852,0.262908


In [52]:
final_data = pd.concat((embedding_df_fr, embedding_df_fw))
final_data = final_data.groupby(final_data.index).mean()

In [53]:
display(final_data)

,new_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0,-0.024952,-0.222366,0.224604,0.302030,0.263212,-0.405940,-0.247950,-0.009713,0.397127,0.223360,0.174874,0.119962,-0.370509,-0.090181,-0.253410,-0.354818,-0.303879,0.059517,-0.202906,-0.548933,-0.104802,0.236403,-0.071516,0.280396,0.023669,0.025529,-0.187288,-0.199713,-0.167799,0.035536,-0.232469,-0.238189
1,1,0.045425,0.153893,0.313686,-0.019985,-0.196837,-0.497429,-0.245584,-0.130554,0.042910,0.093536,0.190061,-0.289323,0.226527,-0.385347,0.140918,0.078269,0.285384,0.195355,0.014456,-0.715068,-0.363537,0.035973,-0.580514,0.224376,0.156259,0.149884,0.264485,-0.476106,0.129328,0.090401,-0.160675,-0.307392
2,2,-0.211608,-0.048720,0.233170,-0.095721,0.019008,-0.673729,-0.186376,0.108787,-0.295669,0.240847,-0.125519,-0.286081,-0.145722,-0.424152,-0.001383,-0.598472,0.396925,-0.048003,-0.415686,-0.370701,0.119518,-0.154070,-0.156009,0.035934,-0.591320,-0.148112,0.076181,-0.476307,0.060270,-0.233346,-0.222473,-0.152664
3,3,-0.264125,-0.048354,0.221413,0.250296,0.050700,-0.695556,-0.214121,-0.276601,0.319021,0.089240,0.038517,-0.173308,0.030227,-0.273812,-0.027520,-0.084102,0.241594,0.524013,-0.108118,-0.087659,-0.004479,0.174143,-0.448422,0.113376,-0.113795,0.217089,-0.143608,0.265923,0.076061,0.388389,-0.315355,-0.107421
4,4,0.140065,-0.520024,-0.159908,0.186898,0.119309,-0.442438,-0.550203,-0.056121,0.596838,0.230975,0.423399,-0.375774,-0.582099,-0.123455,-0.026161,-0.135778,0.449029,0.108778,-0.092896,0.113899,0.306222,-0.118556,-0.199783,-0.152386,0.162577,-0.071473,0.141353,0.092417,0.219046,0.021537,-0.165088,0.040149
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,1759,-0.465017,0.038992,-0.442403,-0.220012,0.392921,0.046240,-0.064529,0.910962,0.220765,0.645999,0.157324,-0.679245,-0.126590,-0.597919,1.230929,-0.498219,0.838593,-0.440109,0.100059,-0.659324,-0.243971,0.068949,-0.340660,-0.250752,-0.105393,0.889527,-0.161363,-0.503576,-0.285870,-0.452497,0.020703,0.267032
1760,1760,-0.548569,-0.663030,0.168672,-0.641771,0.007498,-0.427982,0.378094,-0.061272,-0.427714,0.461209,0.707848,-0.271527,-0.249382,-0.598250,0.266436,-0.582289,0.005334,0.768157,-0.534579,-0.163336,-0.270858,0.117708,-0.957364,0.464780,0.120735,0.173431,0.449871,0.132667,0.075446,-0.061919,-0.928581,-0.535764
1761,1761,-0.906810,0.033241,0.099167,-0.534468,-0.551261,-0.903424,-0.007098,0.303231,-0.251554,0.577695,0.149808,-0.825780,-0.332948,0.081355,0.120388,-0.389623,0.262438,0.472770,0.232752,-0.013581,0.100448,0.190852,-0.097274,-0.235218,0.083243,0.479284,0.463104,0.405101,-0.541159,-0.928349,-0.491973,-0.507083
1762,1762,0.015028,-0.035452,0.167666,-0.131281,-0.372677,-0.074276,-0.036944,0.210421,-0.630576,0.022820,-0.138178,-0.516788,0.050625,-0.006030,0.660804,-0.731573,0.023167,0.576400,0.128687,-0.046356,0.023050,0.394018,-0.318206,0.694861,0.357953,0.677647,-0.371419,0.338643,-0.216629,-0.223026,-0.738672,-0.188209


In [54]:
from sklearn.utils.extmath import softmax
emb_arr = final_data.drop(columns='new_id').to_numpy()

In [55]:
print(emb_arr)

[[-0.02495161 -0.22236559  0.22460371 ...  0.03553613 -0.23246884
  -0.23818867]
 [ 0.04542451  0.15389343  0.31368611 ...  0.09040064 -0.16067493
  -0.30739223]
 [-0.21160847 -0.04871997  0.23317026 ... -0.23334606 -0.22247305
  -0.15266366]
 ...
 [-0.90680993  0.03324139  0.09916678 ... -0.92834866 -0.4919726
  -0.50708282]
 [ 0.01502809 -0.03545205  0.16766597 ... -0.22302639 -0.73867244
  -0.18820898]
 [-0.60104603  0.27600667  0.72244292 ... -0.38403946  0.59749275
  -0.63376999]]


In [56]:
s_arr = softmax(emb_arr)

In [57]:
print(s_arr)

[[0.03143081 0.0258     0.04034001 ... 0.03339066 0.02554065 0.02539498]
 [0.03313184 0.03692776 0.04332614 ... 0.034656   0.0269611  0.02328194]
 [0.02847632 0.03351394 0.04442719 ... 0.027864   0.02816861 0.03020531]
 ...
 [0.01287245 0.03295493 0.03520071 ... 0.01259816 0.0194905  0.0191982 ]
 [0.02996517 0.02849007 0.03490652 ... 0.02361734 0.01410226 0.02445412]
 [0.01505615 0.03619204 0.05655849 ... 0.01870504 0.04991518 0.01457143]]


In [58]:
softmax_df = pd.DataFrame(s_arr, columns=list2)
display(softmax_df)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0.031431,0.025800,0.040340,0.043587,0.041928,0.021473,0.025148,0.031913,0.047936,0.040290,0.038383,0.036332,0.022248,0.029446,0.025011,0.022599,0.023780,0.034201,0.026307,0.018612,0.029019,0.040819,0.030001,0.042655,0.032997,0.033058,0.026721,0.026391,0.027247,0.033391,0.025541,0.025395
1,0.033132,0.036928,0.043326,0.031034,0.026004,0.019253,0.024766,0.027786,0.033049,0.034765,0.038288,0.023706,0.039710,0.021536,0.036452,0.034238,0.042117,0.038491,0.032122,0.015487,0.022011,0.032820,0.017718,0.039624,0.037015,0.036780,0.041246,0.019667,0.036032,0.034656,0.026961,0.023282
2,0.028476,0.033514,0.044427,0.031975,0.035862,0.017939,0.029204,0.039231,0.026180,0.044770,0.031036,0.026433,0.030416,0.023024,0.035139,0.019341,0.052332,0.033538,0.023220,0.024288,0.039654,0.030163,0.030104,0.036475,0.019479,0.030343,0.037973,0.021854,0.037373,0.027864,0.028169,0.030205
3,0.023548,0.029219,0.038267,0.039388,0.032261,0.015296,0.024755,0.023256,0.042190,0.033529,0.031870,0.025787,0.031607,0.023321,0.029834,0.028193,0.039047,0.051789,0.027524,0.028093,0.030529,0.036500,0.019585,0.034348,0.027368,0.038101,0.026564,0.040008,0.033090,0.045221,0.022372,0.027543
4,0.035047,0.018112,0.025964,0.036727,0.034327,0.019574,0.017574,0.028804,0.055338,0.038382,0.046526,0.020923,0.017022,0.026928,0.029680,0.026598,0.047734,0.033967,0.027763,0.034142,0.041382,0.027060,0.024949,0.026160,0.035845,0.028365,0.035092,0.033416,0.037927,0.031129,0.025830,0.031714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,0.017551,0.029053,0.017952,0.022424,0.041390,0.029264,0.026196,0.069483,0.034844,0.053310,0.032702,0.014166,0.024619,0.015367,0.095684,0.016978,0.064632,0.017994,0.030882,0.014452,0.021893,0.029936,0.019875,0.021745,0.025147,0.068010,0.023778,0.016887,0.020994,0.017772,0.028526,0.036494
1760,0.018301,0.016321,0.037494,0.016672,0.031913,0.020646,0.046229,0.029792,0.020652,0.050235,0.064287,0.024143,0.024683,0.017414,0.041345,0.017694,0.031844,0.068283,0.018559,0.026901,0.024159,0.035631,0.012160,0.050415,0.035739,0.037673,0.049669,0.036168,0.034157,0.029773,0.012515,0.018537
1761,0.012872,0.032955,0.035201,0.018680,0.018369,0.012916,0.031652,0.043169,0.024788,0.056803,0.037029,0.013959,0.022850,0.034579,0.035956,0.021591,0.041444,0.051145,0.040232,0.031447,0.035246,0.038581,0.028923,0.025196,0.034645,0.051480,0.050653,0.047799,0.018555,0.012598,0.019490,0.019198
1762,0.029965,0.028490,0.034907,0.025887,0.020335,0.027405,0.028448,0.036431,0.015712,0.030200,0.025709,0.017606,0.031051,0.029341,0.057158,0.014203,0.030210,0.052531,0.033572,0.028181,0.030207,0.043773,0.021473,0.059138,0.042223,0.058128,0.020360,0.041415,0.023769,0.023617,0.014102,0.024454


In [59]:
id_value = final_data['new_id']
print(id_value)

0          0
1          1
2          2
3          3
4          4
        ... 
1759    1759
1760    1760
1761    1761
1762    1762
1763    1763
Name: new_id, Length: 1764, dtype: int64


In [60]:
softmax_df.set_index(keys=id_value, inplace=True)

In [61]:
display(softmax_df)

,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
new_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
0,0.031431,0.025800,0.040340,0.043587,0.041928,0.021473,0.025148,0.031913,0.047936,0.040290,0.038383,0.036332,0.022248,0.029446,0.025011,0.022599,0.023780,0.034201,0.026307,0.018612,0.029019,0.040819,0.030001,0.042655,0.032997,0.033058,0.026721,0.026391,0.027247,0.033391,0.025541,0.025395
1,0.033132,0.036928,0.043326,0.031034,0.026004,0.019253,0.024766,0.027786,0.033049,0.034765,0.038288,0.023706,0.039710,0.021536,0.036452,0.034238,0.042117,0.038491,0.032122,0.015487,0.022011,0.032820,0.017718,0.039624,0.037015,0.036780,0.041246,0.019667,0.036032,0.034656,0.026961,0.023282
2,0.028476,0.033514,0.044427,0.031975,0.035862,0.017939,0.029204,0.039231,0.026180,0.044770,0.031036,0.026433,0.030416,0.023024,0.035139,0.019341,0.052332,0.033538,0.023220,0.024288,0.039654,0.030163,0.030104,0.036475,0.019479,0.030343,0.037973,0.021854,0.037373,0.027864,0.028169,0.030205
3,0.023548,0.029219,0.038267,0.039388,0.032261,0.015296,0.024755,0.023256,0.042190,0.033529,0.031870,0.025787,0.031607,0.023321,0.029834,0.028193,0.039047,0.051789,0.027524,0.028093,0.030529,0.036500,0.019585,0.034348,0.027368,0.038101,0.026564,0.040008,0.033090,0.045221,0.022372,0.027543
4,0.035047,0.018112,0.025964,0.036727,0.034327,0.019574,0.017574,0.028804,0.055338,0.038382,0.046526,0.020923,0.017022,0.026928,0.029680,0.026598,0.047734,0.033967,0.027763,0.034142,0.041382,0.027060,0.024949,0.026160,0.035845,0.028365,0.035092,0.033416,0.037927,0.031129,0.025830,0.031714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,0.017551,0.029053,0.017952,0.022424,0.041390,0.029264,0.026196,0.069483,0.034844,0.053310,0.032702,0.014166,0.024619,0.015367,0.095684,0.016978,0.064632,0.017994,0.030882,0.014452,0.021893,0.029936,0.019875,0.021745,0.025147,0.068010,0.023778,0.016887,0.020994,0.017772,0.028526,0.036494
1760,0.018301,0.016321,0.037494,0.016672,0.031913,0.020646,0.046229,0.029792,0.020652,0.050235,0.064287,0.024143,0.024683,0.017414,0.041345,0.017694,0.031844,0.068283,0.018559,0.026901,0.024159,0.035631,0.012160,0.050415,0.035739,0.037673,0.049669,0.036168,0.034157,0.029773,0.012515,0.018537
1761,0.012872,0.032955,0.035201,0.018680,0.018369,0.012916,0.031652,0.043169,0.024788,0.056803,0.037029,0.013959,0.022850,0.034579,0.035956,0.021591,0.041444,0.051145,0.040232,0.031447,0.035246,0.038581,0.028923,0.025196,0.034645,0.051480,0.050653,0.047799,0.018555,0.012598,0.019490,0.019198


In [62]:
softmax_df['new_id'] = softmax_df.index

In [63]:
softmax_df = softmax_df[ ['new_id'] + [ col for col in softmax_df.columns if col != 'new_id' ] ]
softmax_df.index.name = None

In [64]:
display(softmax_df)

,new_id,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0,0.031431,0.025800,0.040340,0.043587,0.041928,0.021473,0.025148,0.031913,0.047936,0.040290,0.038383,0.036332,0.022248,0.029446,0.025011,0.022599,0.023780,0.034201,0.026307,0.018612,0.029019,0.040819,0.030001,0.042655,0.032997,0.033058,0.026721,0.026391,0.027247,0.033391,0.025541,0.025395
1,1,0.033132,0.036928,0.043326,0.031034,0.026004,0.019253,0.024766,0.027786,0.033049,0.034765,0.038288,0.023706,0.039710,0.021536,0.036452,0.034238,0.042117,0.038491,0.032122,0.015487,0.022011,0.032820,0.017718,0.039624,0.037015,0.036780,0.041246,0.019667,0.036032,0.034656,0.026961,0.023282
2,2,0.028476,0.033514,0.044427,0.031975,0.035862,0.017939,0.029204,0.039231,0.026180,0.044770,0.031036,0.026433,0.030416,0.023024,0.035139,0.019341,0.052332,0.033538,0.023220,0.024288,0.039654,0.030163,0.030104,0.036475,0.019479,0.030343,0.037973,0.021854,0.037373,0.027864,0.028169,0.030205
3,3,0.023548,0.029219,0.038267,0.039388,0.032261,0.015296,0.024755,0.023256,0.042190,0.033529,0.031870,0.025787,0.031607,0.023321,0.029834,0.028193,0.039047,0.051789,0.027524,0.028093,0.030529,0.036500,0.019585,0.034348,0.027368,0.038101,0.026564,0.040008,0.033090,0.045221,0.022372,0.027543
4,4,0.035047,0.018112,0.025964,0.036727,0.034327,0.019574,0.017574,0.028804,0.055338,0.038382,0.046526,0.020923,0.017022,0.026928,0.029680,0.026598,0.047734,0.033967,0.027763,0.034142,0.041382,0.027060,0.024949,0.026160,0.035845,0.028365,0.035092,0.033416,0.037927,0.031129,0.025830,0.031714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,1759,0.017551,0.029053,0.017952,0.022424,0.041390,0.029264,0.026196,0.069483,0.034844,0.053310,0.032702,0.014166,0.024619,0.015367,0.095684,0.016978,0.064632,0.017994,0.030882,0.014452,0.021893,0.029936,0.019875,0.021745,0.025147,0.068010,0.023778,0.016887,0.020994,0.017772,0.028526,0.036494
1760,1760,0.018301,0.016321,0.037494,0.016672,0.031913,0.020646,0.046229,0.029792,0.020652,0.050235,0.064287,0.024143,0.024683,0.017414,0.041345,0.017694,0.031844,0.068283,0.018559,0.026901,0.024159,0.035631,0.012160,0.050415,0.035739,0.037673,0.049669,0.036168,0.034157,0.029773,0.012515,0.018537
1761,1761,0.012872,0.032955,0.035201,0.018680,0.018369,0.012916,0.031652,0.043169,0.024788,0.056803,0.037029,0.013959,0.022850,0.034579,0.035956,0.021591,0.041444,0.051145,0.040232,0.031447,0.035246,0.038581,0.028923,0.025196,0.034645,0.051480,0.050653,0.047799,0.018555,0.012598,0.019490,0.019198
1762,1762,0.029965,0.028490,0.034907,0.025887,0.020335,0.027405,0.028448,0.036431,0.015712,0.030200,0.025709,0.017606,0.031051,0.029341,0.057158,0.014203,0.030210,0.052531,0.033572,0.028181,0.030207,0.043773,0.021473,0.059138,0.042223,0.058128,0.020360,0.041415,0.023769,0.023617,0.014102,0.024454


In [65]:
merged_final_data = pd.merge(nodes_data, softmax_df, how='outer', on='new_id')

In [66]:
merged_final_data = merged_final_data.dropna()

In [67]:
display(merged_final_data)

,new_id,class,tw_id,statuses_count,followers_count,friends_count,favourites_count,listed_count,is_translator,follow_request_sent,protected,verified,notifications,contributors_enabled,following,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31
0,0,False,461277906,10728,379139,2949,1,3406,False,False,False,True,False,False,False,0.031431,0.025800,0.040340,0.043587,0.041928,0.021473,0.025148,0.031913,0.047936,0.040290,0.038383,0.036332,0.022248,0.029446,0.025011,0.022599,0.023780,0.034201,0.026307,0.018612,0.029019,0.040819,0.030001,0.042655,0.032997,0.033058,0.026721,0.026391,0.027247,0.033391,0.025541,0.025395
1,1,False,808280162,141143,26048,337,26670,32,False,False,False,False,False,False,False,0.033132,0.036928,0.043326,0.031034,0.026004,0.019253,0.024766,0.027786,0.033049,0.034765,0.038288,0.023706,0.039710,0.021536,0.036452,0.034238,0.042117,0.038491,0.032122,0.015487,0.022011,0.032820,0.017718,0.039624,0.037015,0.036780,0.041246,0.019667,0.036032,0.034656,0.026961,0.023282
2,2,False,1575510320,180426,120558,108607,454,79,False,False,False,False,False,False,False,0.028476,0.033514,0.044427,0.031975,0.035862,0.017939,0.029204,0.039231,0.026180,0.044770,0.031036,0.026433,0.030416,0.023024,0.035139,0.019341,0.052332,0.033538,0.023220,0.024288,0.039654,0.030163,0.030104,0.036475,0.019479,0.030343,0.037973,0.021854,0.037373,0.027864,0.028169,0.030205
3,3,False,2449235599,13046,26258,18,3889,1,False,False,True,False,False,False,False,0.023548,0.029219,0.038267,0.039388,0.032261,0.015296,0.024755,0.023256,0.042190,0.033529,0.031870,0.025787,0.031607,0.023321,0.029834,0.028193,0.039047,0.051789,0.027524,0.028093,0.030529,0.036500,0.019585,0.034348,0.027368,0.038101,0.026564,0.040008,0.033090,0.045221,0.022372,0.027543
4,4,False,1113769100,72434,75208,78570,13374,35,False,False,False,False,False,False,False,0.035047,0.018112,0.025964,0.036727,0.034327,0.019574,0.017574,0.028804,0.055338,0.038382,0.046526,0.020923,0.017022,0.026928,0.029680,0.026598,0.047734,0.033967,0.027763,0.034142,0.041382,0.027060,0.024949,0.026160,0.035845,0.028365,0.035092,0.033416,0.037927,0.031129,0.025830,0.031714
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,1759,True,355303278,29159,1679,1619,12974,3,False,False,False,False,False,False,False,0.017551,0.029053,0.017952,0.022424,0.041390,0.029264,0.026196,0.069483,0.034844,0.053310,0.032702,0.014166,0.024619,0.015367,0.095684,0.016978,0.064632,0.017994,0.030882,0.014452,0.021893,0.029936,0.019875,0.021745,0.025147,0.068010,0.023778,0.016887,0.020994,0.017772,0.028526,0.036494
1760,1760,True,288948026,12138,903,692,1843,0,False,False,False,False,False,False,False,0.018301,0.016321,0.037494,0.016672,0.031913,0.020646,0.046229,0.029792,0.020652,0.050235,0.064287,0.024143,0.024683,0.017414,0.041345,0.017694,0.031844,0.068283,0.018559,0.026901,0.024159,0.035631,0.012160,0.050415,0.035739,0.037673,0.049669,0.036168,0.034157,0.029773,0.012515,0.018537
1761,1761,True,2183080705,16011,1289,4149,11653,25,False,False,False,False,False,False,False,0.012872,0.032955,0.035201,0.018680,0.018369,0.012916,0.031652,0.043169,0.024788,0.056803,0.037029,0.013959,0.022850,0.034579,0.035956,0.021591,0.041444,0.051145,0.040232,0.031447,0.035246,0.038581,0.028923,0.025196,0.034645,0.051480,0.050653,0.047799,0.018555,0.012598,0.019490,0.019198
1762,1762,True,859364185,22361,896,305,16198,24,False,False,False,False,False,False,False,0.029965,0.028490,0.034907,0.025887,0.020335,0.027405,0.028448,0.036431,0.015712,0.030200,0.025709,0.017606,0.031051,0.029341,0.057158,0.014203,0.030210,0.052531,0.033572,0.028181,0.030207,0.043773,0.021473,0.059138,0.042223,0.058128,0.020360,0.041415,0.023769,0.023617,0.014102,0.024454


In [68]:
label = merged_final_data['class']
merged_final_data = merged_final_data.drop(columns='class')
merged_final_data.insert(loc=46, column='label', value=label)
merged_final_data = merged_final_data.drop(columns=['new_id', 'tw_id'])

In [69]:
display(merged_final_data)

,statuses_count,followers_count,friends_count,favourites_count,listed_count,is_translator,follow_request_sent,protected,verified,notifications,contributors_enabled,following,f0,f1,f2,f3,f4,f5,f6,f7,f8,f9,f10,f11,f12,f13,f14,f15,f16,f17,f18,f19,f20,f21,f22,f23,f24,f25,f26,f27,f28,f29,f30,f31,label
0,10728,379139,2949,1,3406,False,False,False,True,False,False,False,0.031431,0.025800,0.040340,0.043587,0.041928,0.021473,0.025148,0.031913,0.047936,0.040290,0.038383,0.036332,0.022248,0.029446,0.025011,0.022599,0.023780,0.034201,0.026307,0.018612,0.029019,0.040819,0.030001,0.042655,0.032997,0.033058,0.026721,0.026391,0.027247,0.033391,0.025541,0.025395,False
1,141143,26048,337,26670,32,False,False,False,False,False,False,False,0.033132,0.036928,0.043326,0.031034,0.026004,0.019253,0.024766,0.027786,0.033049,0.034765,0.038288,0.023706,0.039710,0.021536,0.036452,0.034238,0.042117,0.038491,0.032122,0.015487,0.022011,0.032820,0.017718,0.039624,0.037015,0.036780,0.041246,0.019667,0.036032,0.034656,0.026961,0.023282,False
2,180426,120558,108607,454,79,False,False,False,False,False,False,False,0.028476,0.033514,0.044427,0.031975,0.035862,0.017939,0.029204,0.039231,0.026180,0.044770,0.031036,0.026433,0.030416,0.023024,0.035139,0.019341,0.052332,0.033538,0.023220,0.024288,0.039654,0.030163,0.030104,0.036475,0.019479,0.030343,0.037973,0.021854,0.037373,0.027864,0.028169,0.030205,False
3,13046,26258,18,3889,1,False,False,True,False,False,False,False,0.023548,0.029219,0.038267,0.039388,0.032261,0.015296,0.024755,0.023256,0.042190,0.033529,0.031870,0.025787,0.031607,0.023321,0.029834,0.028193,0.039047,0.051789,0.027524,0.028093,0.030529,0.036500,0.019585,0.034348,0.027368,0.038101,0.026564,0.040008,0.033090,0.045221,0.022372,0.027543,False
4,72434,75208,78570,13374,35,False,False,False,False,False,False,False,0.035047,0.018112,0.025964,0.036727,0.034327,0.019574,0.017574,0.028804,0.055338,0.038382,0.046526,0.020923,0.017022,0.026928,0.029680,0.026598,0.047734,0.033967,0.027763,0.034142,0.041382,0.027060,0.024949,0.026160,0.035845,0.028365,0.035092,0.033416,0.037927,0.031129,0.025830,0.031714,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1759,29159,1679,1619,12974,3,False,False,False,False,False,False,False,0.017551,0.029053,0.017952,0.022424,0.041390,0.029264,0.026196,0.069483,0.034844,0.053310,0.032702,0.014166,0.024619,0.015367,0.095684,0.016978,0.064632,0.017994,0.030882,0.014452,0.021893,0.029936,0.019875,0.021745,0.025147,0.068010,0.023778,0.016887,0.020994,0.017772,0.028526,0.036494,True
1760,12138,903,692,1843,0,False,False,False,False,False,False,False,0.018301,0.016321,0.037494,0.016672,0.031913,0.020646,0.046229,0.029792,0.020652,0.050235,0.064287,0.024143,0.024683,0.017414,0.041345,0.017694,0.031844,0.068283,0.018559,0.026901,0.024159,0.035631,0.012160,0.050415,0.035739,0.037673,0.049669,0.036168,0.034157,0.029773,0.012515,0.018537,True
1761,16011,1289,4149,11653,25,False,False,False,False,False,False,False,0.012872,0.032955,0.035201,0.018680,0.018369,0.012916,0.031652,0.043169,0.024788,0.056803,0.037029,0.013959,0.022850,0.034579,0.035956,0.021591,0.041444,0.051145,0.040232,0.031447,0.035246,0.038581,0.028923,0.025196,0.034645,0.051480,0.050653,0.047799,0.018555,0.012598,0.019490,0.019198,True
1762,22361,896,305,16198,24,False,False,False,False,False,False,False,0.029965,0.028490,0.034907,0.025887,0.020335,0.027405,0.028448,0.036431,0.015712,0.030200,0.025709,0.017606,0.031051,0.029341,0.057158,0.014203,0.030210,0.052531,0.033572,0.028181,0.030207,0.043773,0.021473,0.059138,0.042223,0.058128,0.020360,0.041415,0.023769,0.023617,0.014102,0.024454,True


In [70]:
X = merged_final_data.iloc[:,:-1]

y = merged_final_data.iloc[:,-1]

In [71]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [72]:
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [73]:
scaled_df = pd.DataFrame(X_train)
display(scaled_df)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43
0,0.792197,-0.173502,-0.173711,2.142408,-0.200575,-0.037676,0.0,-0.217975,-0.674269,0.0,0.0,0.0,-0.322934,-0.007971,0.644030,-0.132033,0.140164,0.990086,-0.225336,0.023889,-1.587074,-0.973256,0.358928,-0.575079,1.620291,-0.003568,-0.256499,-0.079029,1.316756,0.664662,-0.709638,-0.386424,0.955222,-0.119882,0.236766,0.312546,-0.394162,0.083841,-0.430863,0.479183,-0.505516,-0.292134,-0.187979,-0.341428
1,0.607011,0.063928,0.074662,-0.343855,-0.113457,-0.037676,0.0,-0.217975,-0.674269,0.0,0.0,0.0,1.037113,-0.000268,1.161790,-0.860945,-0.700689,1.056589,-0.033016,0.344420,1.297954,0.338262,-1.172385,-0.076313,-1.467160,-0.907466,-0.160930,0.176571,0.365460,-0.525182,-0.947254,-0.002893,-0.551807,-1.047504,0.735505,0.646705,-0.902990,-0.905916,-1.085360,-1.184424,2.203605,-0.312873,0.712935,0.106385
2,-0.202126,-0.173519,-0.169751,-0.373000,-0.196844,-0.037676,0.0,-0.217975,-0.674269,0.0,0.0,0.0,-0.790652,1.831455,-0.163649,0.426819,0.063015,-0.729968,0.390392,-0.689219,1.202070,-0.879645,-0.998172,-0.549327,-0.643502,-0.499778,-0.388829,-0.609837,0.262522,0.088756,-0.426011,-0.562270,5.002151,-0.133687,3.098717,-1.172593,-1.398201,-1.110729,0.138458,0.367587,0.760932,-0.107753,-0.255696,-1.022274
3,-0.550329,-0.173604,-0.185657,-0.373563,-0.204927,-0.037676,0.0,-0.217975,-0.674269,0.0,0.0,0.0,0.971086,1.915048,-0.688076,-1.137221,-0.196276,-1.318379,-0.479210,2.016747,-0.954498,0.991951,-0.538971,1.106986,-0.208453,-0.815504,-0.027513,0.904503,-0.291217,-0.931969,0.032425,-1.027733,-0.542656,-0.516419,-0.641433,-0.714856,0.161360,-1.180768,-0.454783,-0.767111,1.807939,-0.422630,-0.706801,1.774522
4,-0.100101,-0.148639,-0.165939,0.635377,-0.148155,-0.037676,0.0,-0.217975,-0.674269,0.0,0.0,0.0,-0.414978,-1.672547,0.121600,2.778694,-0.671653,0.154084,-0.968791,0.980965,1.111743,1.454241,0.344366,-0.625568,-0.514541,-0.166358,-0.491648,0.819115,0.398864,-1.145295,-0.131333,-1.251754,-0.591032,-0.359328,-1.251936,-0.432773,-0.635510,-1.141994,0.461601,0.312212,-0.789602,-0.091680,0.590892,4.013870
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1406,-0.535493,-0.149839,-0.178632,-0.372484,-0.108047,-0.037676,0.0,-0.217975,1.483087,0.0,0.0,0.0,-1.446927,1.286837,1.249392,-0.501675,0.599052,1.411953,-0.241193,-0.303918,0.714722,-0.486096,-0.911882,0.325176,-0.611538,1.230106,-0.124689,-1.202187,-0.685062,-0.395711,-0.787414,0.243695,-0.292976,-0.888129,0.181595,0.012317,-0.669582,0.767276,-1.243832,2.867966,-1.024705,0.843909,0.805881,0.557486
1407,-0.426375,-0.167909,-0.088197,0.720678,-0.199642,-0.037676,0.0,-0.217975,-0.674269,0.0,0.0,0.0,-0.863451,3.200121,-0.321617,0.550816,0.334545,-0.691494,0.355702,-0.463107,1.873633,-0.910228,-0.460090,-0.273911,0.163237,0.350556,-0.143473,-0.334519,0.363488,-0.854804,-0.432613,-0.515395,3.003548,0.827181,0.665794,-0.872748,-0.964743,-1.052534,-0.001319,-0.487434,0.145872,0.420812,-0.878161,-0.722298
1408,0.977516,0.165395,-0.185306,-0.373469,-0.103384,-0.037676,0.0,-0.217975,1.483087,0.0,0.0,0.0,0.038391,0.547030,-0.348452,-0.217777,-1.064838,2.910544,1.257430,-0.441360,-0.508167,0.125883,2.845903,0.096666,-1.443139,0.950525,0.290702,-1.099496,1.041294,-0.937621,-0.893431,-1.059238,-0.110295,0.089644,-1.010598,-0.804205,0.722226,-1.187721,-0.396593,-0.182392,0.208710,0.614591,1.352885,-0.125881
1409,-0.534494,-0.163845,0.496415,0.023776,-0.203559,-0.037676,0.0,-0.217975,-0.674269,0.0,0.0,0.0,-0.009105,-0.872242,-0.210717,-0.729479,3.614158,0.039327,-0.190082,-1.074205,-0.942822,-0.687190,1.120501,-0.832648,-0.078826,-0.712266,-0.184403,-1.337973,-0.207996,1.251669,0.168182,0.267472,1.743928,-0.789289,1.001089,-1.122740,1.360495,-0.531443,0.961504,-0.560900,0.828090,-0.638669,-0.775435,1.257487


In [74]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=20, random_state=0)
clf.fit(X_train, y_train)
y_pred = clf.predict(X_test)

In [75]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))
print(accuracy_score(y_test, y_pred))

[[173  25]
 [ 29 126]]
              precision    recall  f1-score   support

       False       0.86      0.87      0.86       198
        True       0.83      0.81      0.82       155

    accuracy                           0.85       353
   macro avg       0.85      0.84      0.84       353
weighted avg       0.85      0.85      0.85       353

0.8470254957507082


In [76]:
from sklearn.model_selection import GridSearchCV

rfc = RandomForestClassifier(random_state=42)

param_grid = { 
    'n_estimators': [50, 150, 250],
    'max_features': ['auto', 'sqrt',  0.5, 1.0],
    'min_samples_split' : [2, 4, 6],
}


In [77]:
CV_rfc = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

In [78]:
CV_rfc.best_params_

{'max_features': 0.5, 'min_samples_split': 4, 'n_estimators': 250}

In [79]:
rfc1=RandomForestClassifier(random_state=42, max_features=0.5, n_estimators= 250, min_samples_split=4)

In [80]:
rfc1.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=4,
                       min_weight_fraction_leaf=0.0, n_estimators=250,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [81]:
pred = rfc1.predict(X_test)

In [82]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred))

Accuracy for Random Forest on CV data:  0.8980169971671388


In [83]:
pd.set_option('display.max_rows', 100)

In [84]:
feature_imp = pd.Series(clf.feature_importances_,index=X.columns).sort_values(ascending=False)
feature_imp

statuses_count          0.126474
followers_count         0.053841
f9                      0.049160
favourites_count        0.043962
f19                     0.041526
f16                     0.040384
f10                     0.034688
f15                     0.031631
f23                     0.030180
f1                      0.029430
f3                      0.028554
listed_count            0.028484
f7                      0.027618
f14                     0.026528
f31                     0.023598
f4                      0.022685
f20                     0.020983
f8                      0.020019
f17                     0.019476
f26                     0.018673
f5                      0.018490
f27                     0.018073
friends_count           0.018069
f24                     0.017982
f0                      0.017698
f28                     0.017565
f12                     0.017193
f18                     0.017057
f21                     0.016482
f6                      0.016416
f13       

In [93]:
from sklearn.feature_selection import SelectFromModel
feat_sel = SelectFromModel(clf,threshold=0.0144)
feat_sel.fit(X_train, y_train)

SelectFromModel(estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                                 class_weight=None,
                                                 criterion='gini',
                                                 max_depth=None,
                                                 max_features='auto',
                                                 max_leaf_nodes=None,
                                                 max_samples=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=20, n_jobs=None,
                                                 oob_score=False,


In [94]:
X_imp_train = feat_sel.transform(X_train)
X_imp_test = feat_sel.transform(X_test)

In [95]:
CV_rfc_imp = GridSearchCV(estimator=rfc, param_grid=param_grid, cv=5)
CV_rfc_imp.fit(X_imp_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=42,
                                  

In [96]:
CV_rfc_imp.best_params_

{'max_features': 0.5, 'min_samples_split': 6, 'n_estimators': 150}

In [97]:
rfc_imp = RandomForestClassifier(random_state=42, max_features=0.5, n_estimators=150, min_samples_split=6)

In [98]:
rfc_imp.fit(X_imp_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.5,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=6,
                       min_weight_fraction_leaf=0.0, n_estimators=150,
                       n_jobs=None, oob_score=False, random_state=42, verbose=0,
                       warm_start=False)

In [99]:
pred_imp = rfc_imp.predict(X_imp_test)

In [100]:
print("Accuracy for Random Forest on CV data: ",accuracy_score(y_test,pred_imp))

Accuracy for Random Forest on CV data:  0.8923512747875354
